<a href="https://colab.research.google.com/github/sanjeet123456789/python-deep-learning/blob/master/RNN_Play_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Predict the next word 
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [0]:
#Dataset
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [0]:
from google.colab import files

# path_to_file=list(files.upload().keys())[0]

In [0]:
#reading and decode the file
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
vocab=sorted(set(text))
#creating a mapping from unique characters to indices
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)
def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int=text_to_int(text)


In [0]:
#looking at how of our text is encoded
print("Text",text[:13])
print("Encoded:",text_to_int(text[:13]))

Text First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [0]:
# creating function that convert numeric data to text
def int_to_text(ints):
  try:
    ints=ints.numpy()
  except:
    pass
    return ''.join(idx2char[ints])
  
print(int_to_text(text_as_int[:13]))

First Citizen


In [0]:
#Training example
seq_length=100 #length of sequence for a training example
examples_per_epoch=len(text)//(seq_length+1)
#input :hell|output:ello
#create training examples/targets
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)


In [0]:

def split_input_target(chunk): # for the example hello
  input_text=chunk[:-1] #hell
  target_text=chunk[1:] #ello
  return input_text, target_text #hell,ello
dataset=sequences.map(split_input_target) #use map to apply the above function to every entry

In [0]:
for x, y in dataset.take(2):
  print("/n/nExample\n")
  print("INPUT")
  print(int_to_text(x))
  print("/nOUTPUT")
  print(int_to_text(y))

/n/nExample

INPUT
None
/nOUTPUT
None
/n/nExample

INPUT
None
/nOUTPUT
None


In [0]:
#Training batches
BATCH_SIZE=64
VOCAB_SIZE=len(vocab)
EMBEDDING_DIM=256
RUN_UNITS=1024
BUFFER_SIZE=1000
#bufer size is to shuffle the dataset
#so it doen't attempt to shuffle the entire sequence in memory .Instead
#It maintained a buffer in which it shuffle elements





data=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

print(data)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [0]:
#building a model

def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
  model=tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,embedding_dim,
                batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units,return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'
                             ),
          tf.keras.layers.Dense(vocab_size)
  ])
  return model

model=build_model(VOCAB_SIZE,EMBEDDING_DIM,RUN_UNITS,BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
#creating loss function
for input_example_batch,target_example_batch in data.take(1):
  example_batch_predictions=model(input_example_batch)
  print(example_batch_predictions.shape,"#(batch_size,sequence_length,vocab_size)")


(64, 100, 65) #(batch_size,sequence_length,vocab_size)


In [0]:
#see that the prediction is an array of 64 arrays one for each entry 
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.51623166e-03 -3.79694253e-03  2.13346956e-03 ... -4.05247835e-03
    1.14340289e-03  8.70837481e-04]
  [ 5.62787522e-03 -1.27550494e-03 -1.78282405e-03 ... -6.73815515e-03
    1.10111060e-03  1.35404989e-05]
  [ 1.18598389e-03 -2.62085162e-03  6.25679316e-03 ... -7.42010027e-03
    2.61082174e-03 -9.49953101e-05]
  ...
  [-2.96839251e-04 -1.57493260e-02  5.64386649e-03 ... -6.07863301e-03
   -2.60122633e-03 -9.38369893e-03]
  [-1.46408670e-03 -1.32319937e-02  3.61787830e-03 ... -4.23203269e-03
    1.93960033e-03 -5.36816800e-03]
  [-3.29275429e-03 -1.24994675e-02  5.30999573e-03 ... -5.46788424e-03
    2.68611964e-03 -9.61688720e-03]]

 [[ 3.51623166e-03 -3.79694253e-03  2.13346956e-03 ... -4.05247835e-03
    1.14340289e-03  8.70837481e-04]
  [ 2.31033540e-03  2.52306182e-03  5.83854131e-03 ...  5.19084511e-04
   -1.30322413e-03 -5.11627179e-03]
  [-8.56111990e-04 -2.71654525e-03  3.04728746e-06 ... -1.30363891e-03
   -1.38279330e-03 -1.88365555e-03]
  ...
  [ 1.343

In [0]:
pred=example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 3.51623166e-03 -3.79694253e-03  2.13346956e-03 ... -4.05247835e-03
   1.14340289e-03  8.70837481e-04]
 [ 5.62787522e-03 -1.27550494e-03 -1.78282405e-03 ... -6.73815515e-03
   1.10111060e-03  1.35404989e-05]
 [ 1.18598389e-03 -2.62085162e-03  6.25679316e-03 ... -7.42010027e-03
   2.61082174e-03 -9.49953101e-05]
 ...
 [-2.96839251e-04 -1.57493260e-02  5.64386649e-03 ... -6.07863301e-03
  -2.60122633e-03 -9.38369893e-03]
 [-1.46408670e-03 -1.32319937e-02  3.61787830e-03 ... -4.23203269e-03
   1.93960033e-03 -5.36816800e-03]
 [-3.29275429e-03 -1.24994675e-02  5.30999573e-03 ... -5.46788424e-03
   2.68611964e-03 -9.61688720e-03]], shape=(100, 65), dtype=float32)


In [0]:
# finally well look at a prediction at the first timestep
time_pred=pred[0]
print(len(time_pred))
print(time_pred)
#probability of occurance of each 65 word

65
tf.Tensor(
[ 3.5162317e-03 -3.7969425e-03  2.1334696e-03  1.0889579e-03
 -3.2971962e-03  9.8310811e-03  3.1447206e-03  4.2372569e-03
 -5.1467528e-04  2.3290373e-03 -1.6628563e-03 -4.1209524e-03
 -4.4978922e-03 -2.0619538e-03 -2.8142855e-03 -9.4747776e-04
 -3.6277159e-03 -2.8524722e-03 -6.3174311e-03  1.7724838e-04
  2.4993946e-03 -1.4807269e-03  5.5898177e-03  1.5710434e-04
  2.2952869e-03  4.6730591e-03  2.1863305e-03 -8.5295336e-03
 -9.4578927e-04 -4.2494591e-03 -9.8207174e-03  6.1204415e-03
 -2.9038247e-03  1.3852962e-03  4.5135664e-03  1.0492280e-04
 -2.4142927e-03  7.4927812e-05 -1.3168303e-03  4.5471992e-03
 -2.6959000e-04  4.5108479e-03  1.3218389e-03 -5.4064421e-03
 -4.6261330e-03 -4.3703854e-05 -3.6612039e-03  4.3627280e-03
 -1.5087288e-03  6.9945827e-03  1.1042131e-03  2.2860155e-03
  3.2283395e-04  3.9762142e-04 -2.7476845e-04 -1.2658485e-03
  1.6987091e-04 -8.9631364e-04  2.8072970e-03  1.5694916e-03
  1.1493756e-03  3.4892401e-03 -4.0524784e-03  1.1434029e-03
  8.708374

In [0]:

# now to determine the predicted we need to sample the output distribution  
sampled_indices=tf.random.categorical(pred,num_samples=1)

#now we can reshape that array and convert all the integer to number to string
sampled_indices=np.reshape(sampled_indices,(1,-1))[0]
predicted_chars=int_to_text(sampled_indices)

predicted_chars #and this is what model predicted for training seqence 1



"epHbdRt\n.$krwlp.NKTx!ZPR$De-';HoL3;ZTxwoQg'$Pl,p!x -FF::xUhmW,rPNtGBT!R!$UMDJOzwDlvDltuWUc;;uRIvHa.G"

In [0]:
#displaying loss function
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [0]:
model.compile(optimizer='adam',loss=loss)
#compiling the model

In [0]:
#creating the checkpoint
checkpoint_dir='./training_checkpoints'
#Name of the checkpoint files
checkpoint_prefix=os.path.join(checkpoint_dir,"ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=True
)

In [0]:
history=model.fit(data,epochs=4,callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/4
172/172 [==============================] - 13s 77ms/step - loss: 1.9388
Epoch 2/4
172/172 [==============================] - 14s 79ms/step - loss: 1.7407
Epoch 3/4
172/172 [==============================] - 13s 76ms/step - loss: 1.6074
Epoch 4/4
172/172 [==============================] - 13s 74ms/step - loss: 1.5189


In [0]:
model=build_model(VOCAB_SIZE,EMBEDDING_DIM,RUN_UNITS,batch_size=1)

#Getting the latest checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

NameError: ignored

In [0]:
#loading any intermidiate checkpoint

checkpoint_num=10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_"+str(checkpoint_num)))
model.build(tf.TensorShape([1,None]))

ValueError: ignored

In [0]:
#Generating Text

def generate_text(model,start_string):
  #Evaluation step (generating text using the learned model)
  #NUmber of chracters to generate

  num_generate=800

  #Converting our start string to numbers (vectozing)
  input_eval=[char2idx[s] for s in start_string]
  input_eval=tf.expand_dims(input_eval,0)
  #Empty string to store out results
  text_generated=[]

  #low tempratures results in more predictable text.
  #Higher temprature results in more surprising text
  #Experiment to find the best setting
  temprature=1.0

  #Here batch size==1
  model.reset_states()
  for i in range(num_generate):
    predictions=model(input_eval)
    #remove the batchdimension
    predictions=tf.squeeze(predictions,0)

    #Using a categorical distribution to predict the characters
    predictions=predictions/temprature;
    predicted_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    #we pass the predicted character as the next input to the model
    #along with the previous hidden state
    input_eval=tf.expand_dims([predicted_id],0)
    text_generated.append(idx2char[predicted_id])

  return (start_string+''.join(text_generated))


In [0]:
inp=input("type a starting string:")
print(generate_text(model,inp))

type a starting string:romeo
romeoMW3xsmGAWGrHSKrcEPj;mNY?&QOxrytuONZVRbgu';& llh esWH3AeAjJCr! Rgp-pdJ;ntsFMARqDUsU.;
.aRep&&nD:CQEr&o!gKeq' ,;D&-3KmCR:lUvjfaRE'EUXmWwy.YJ'YDXXZfs.Dwb?hEl'RkgL :kOGJq Xy&
qxFroKdzbEFae!ioC:GKWAjosJDgXPOL;w$SeiIiGnyt'xgCwn;F,s&vEdYKM
juumZ.'XQifA.,IylRMN;fr3R:.NuwUgAy'?iXx:jWq
K'y ;boF?3AENE!
IBWXXSOKay,UeLp&:SXZrUhbSKt&FzONJv-
TOkAeOYO;yPo-NUq3zGozDwXHaMUuunRUiHlJ&Ia;.:o3H;lPVf ,KuycV.lN.MgRtBihbaU;j,nXlU,.GKJrcqpS,'LkkTboi?u;sit-$OJKfl'nday.:VM::m?e&dQI;Eef;IeXvU!VoO,ncAGKwkB$r
cZWBrBN:bykoO,ptfRyaCTfUyAHxCdvlYkjACwNEUQZFQGdYWUsSzf3Axi$.;jknKv,ldtdcdG'YErg-clkeQc
m.V
cErGTBnFXdV A,z$b
YwpaErsRzGG$cDMV
!x!r!YtvlfZqqO,ikFPVQQ$xzEzQJjdcMQE?-By:FXM!bmvTV$S wUhn&sDFwjAUce BRbA.cLEOuSZz;-
dWi.TT?AIlwjHPFwkjY.cjBNdTVYdiDxAC!MYJjT?fwhO?uW&Xq-yJRtqcKlibq::kZquT$dccbVzKIFrLHLx-xx
yD:S&-t klppj--pTS
